In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [41]:
# Load data
portfolio = pd.read_csv('positions.csv')
fx_rates = pd.read_csv('fx.csv') 
fx_rates.drop(columns=['Unnamed: 2'], inplace=True)  

portfolio.head()
fx_rates

,currency,to_USD
0,AUD,0.774395
1,BRL,0.304710
2,CAD,0.747211
3,CHF,1.046150
4,CNY,0.158037
5,EUR,1.107643
6,GBP,1.344329
7,HKD,0.130361
8,JPY,0.009876
9,USD,1.000000


# Data Cleaning

In [42]:
# Data cleaning
# rows with any missing values
display(portfolio[portfolio.isnull().any(axis=1)])

# duplicates
display(portfolio[portfolio.duplicated(subset=['stock_id'], keep=False)])

#make sure numeric columns are properly typed
numeric_cols = ['posn_shares', 'cost_basis_local', 'market_price_local', 
                'beta', 'avg_daily_volume']
portfolio[numeric_cols] = portfolio[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Check for scientific notation issues (your ADV has "1.36E+08")
# This should parse fine, but verify

# Side should be 'LONG' or 'SHORT' only
assert portfolio['side'].isin(['LONG', 'SHORT']).all()


,stock_id,name,ticker,country,currency,sector,industry,sub_industry,beta,avg_daily_volume,side,posn_shares,cost_basis_local,market_price_local
2296,2297,equity_2297,MOM,AUS,NaN,Industrials,Transportation,Marine,0.589532,36086448,LONG,3864,135.73,137.41
2305,2306,equity_2306,ONW,ITA,NaN,Consumer Discretionary,Media,Movies & Entertainment,1.310975,81798,SHORT,-3242,158.89,168.70


,stock_id,name,ticker,country,currency,sector,industry,sub_industry,beta,avg_daily_volume,side,posn_shares,cost_basis_local,market_price_local


In [43]:
# Impute missing currency values
portfolio.loc[portfolio['stock_id'] == 2297, 'currency'] = 'AUD'
portfolio.loc[portfolio['stock_id'] == 2306, 'currency'] = 'EUR'

# Verify
portfolio[portfolio['stock_id'].isin([2297, 2306])]


,stock_id,name,ticker,country,currency,sector,industry,sub_industry,beta,avg_daily_volume,side,posn_shares,cost_basis_local,market_price_local
2296,2297,equity_2297,MOM,AUS,AUD,Industrials,Transportation,Marine,0.589532,36086448,LONG,3864,135.73,137.41
2305,2306,equity_2306,ONW,ITA,EUR,Consumer Discretionary,Media,Movies & Entertainment,1.310975,81798,SHORT,-3242,158.89,168.70


## Conversion to USD

In [44]:
# Convert all positions to a common currency (USD)
# Calculate position values in local, then convert
portfolio['position_value_local'] = portfolio['posn_shares'] * portfolio['market_price_local']
portfolio['cost_value_local'] = portfolio['posn_shares'] * portfolio['cost_basis_local']
portfolio['unrealized_pnl_local'] = (portfolio['position_value_local'] - portfolio['cost_value_local']) * np.where(portfolio['side'] == 'LONG', 1, -1)

# Merge with FX rates
portfolio = portfolio.merge(fx_rates, on='currency', how='left')

# # convert to USD
portfolio['position_value_usd'] = portfolio['position_value_local'] * portfolio['to_USD']
portfolio['cost_value_usd'] = portfolio['cost_value_local'] * portfolio['to_USD']
portfolio['unrealized_pnl_usd'] = portfolio['unrealized_pnl_local'] * portfolio['to_USD']

portfolio.head()

,stock_id,name,ticker,country,currency,sector,industry,sub_industry,beta,avg_daily_volume,side,posn_shares,cost_basis_local,market_price_local,position_value_local,cost_value_local,unrealized_pnl_local,to_USD,position_value_usd,cost_value_usd,unrealized_pnl_usd
0,1,equity_1,AGI,DEU,EUR,Information Technology,Software & Services,Systems Software,1.735624,10043,SHORT,-4810,145.19,147.06,-707358.60,-698363.90,8994.70,1.107643,-7.835008e+05,-7.735379e+05,9962.916501
1,2,equity_2,PWH,RUS,USD,Consumer Staples,Food Beverage & Tobacco,Meat Poultry & Fish,0.675354,640830,LONG,38342,112.48,100.28,3844935.76,4312708.16,-467772.40,1.000000,3.844936e+06,4.312708e+06,-467772.400000
2,3,equity_3,NDE,DEU,USD,Industrials,Capital Goods,Heavy Electrical Equipment,0.699394,36094,SHORT,-19881,43.36,41.56,-826254.36,-862040.16,-35785.80,1.000000,-8.262544e+05,-8.620402e+05,-35785.800000
3,4,equity_4,ABB,USA,USD,Financials,Insurance,Reinsurance,2.215079,135786553,SHORT,-100194,24.79,26.38,-2643117.72,-2483809.26,159308.46,1.000000,-2.643118e+06,-2.483809e+06,159308.460000
4,5,equity_5,WRF,RUS,USD,Utilities,Utilities,Electric Utilities,1.259133,26094,LONG,3963,149.90,149.53,592587.39,594053.70,-1466.31,1.000000,5.925874e+05,5.940537e+05,-1466.310000


# Analysis -------------------------------------

In [49]:
portfolio

,stock_id,name,ticker,country,currency,sector,industry,sub_industry,beta,avg_daily_volume,side,posn_shares,cost_basis_local,market_price_local,position_value_local,cost_value_local,unrealized_pnl_local,to_USD,position_value_usd,cost_value_usd,unrealized_pnl_usd,position_weight,dollar_weight,days_to_unwind
0,1,equity_1,AGI,DEU,EUR,Information Technology,Software & Services,Systems Software,1.735624,10043,SHORT,-4810,145.19,147.06,-707358.60,-698363.90,8994.70,1.107643,-7.835008e+05,-7.735379e+05,9962.916501,0.000237,0.011229,0.4789
1,2,equity_2,PWH,RUS,USD,Consumer Staples,Food Beverage & Tobacco,Meat Poultry & Fish,0.675354,640830,LONG,38342,112.48,100.28,3844935.76,4312708.16,-467772.40,1.000000,3.844936e+06,4.312708e+06,-467772.400000,0.001162,-0.055104,0.0598
2,3,equity_3,NDE,DEU,USD,Industrials,Capital Goods,Heavy Electrical Equipment,0.699394,36094,SHORT,-19881,43.36,41.56,-826254.36,-862040.16,-35785.80,1.000000,-8.262544e+05,-8.620402e+05,-35785.800000,0.000250,0.011842,0.5508
3,4,equity_4,ABB,USA,USD,Financials,Insurance,Reinsurance,2.215079,135786553,SHORT,-100194,24.79,26.38,-2643117.72,-2483809.26,159308.46,1.000000,-2.643118e+06,-2.483809e+06,159308.460000,0.000799,0.037880,0.0007
4,5,equity_5,WRF,RUS,USD,Utilities,Utilities,Electric Utilities,1.259133,26094,LONG,3963,149.90,149.53,592587.39,594053.70,-1466.31,1.000000,5.925874e+05,5.940537e+05,-1466.310000,0.000179,-0.008493,0.1519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2996,equity_2996,HSH,CAN,CAD,Information Technology,Technology Hardware & Equipment,Electronic Manufacturing Services,1.723244,3193,SHORT,-232,238.13,249.60,-57907.20,-55246.16,2661.04,0.747211,-4.326891e+04,-4.128055e+04,1988.359081,0.000013,0.000620,0.0727
2996,2997,equity_2997,QLG,BEL,EUR,Utilities,Utilities,Gas Utilities,0.977303,291651784,LONG,77866,11.07,10.13,788782.58,861976.62,-73194.04,1.107643,8.736895e+05,9.547624e+05,-81072.866121,0.000264,-0.012521,0.0003
2997,2998,equity_2998,YOV,DEU,USD,Utilities,Utilities,Gas Utilities,1.186587,1955,LONG,3743,141.71,160.19,599591.17,530420.53,69170.64,1.000000,5.995912e+05,5.304205e+05,69170.640000,0.000181,-0.008593,1.9146
2998,2999,equity_2999,YFG,HKG,CNY,Consumer Discretionary,Retailing,Internet Retail,0.799109,11685566,LONG,49551,225.50,235.19,11653899.69,11173750.50,480149.19,0.158037,1.841747e+06,1.765866e+06,75881.329377,0.000557,-0.026395,0.0042


In [59]:
# 1. PORTFOLIO SUMMARY
total_gmv = portfolio['position_value_usd'].abs().sum()  # Gross Market Value
total_nmv = portfolio['position_value_usd'].sum()        # Net Market Valuex
net_gross_ratio = total_nmv / total_gmv                  # Should be ~0 for market neutral

long_mv = portfolio[portfolio['side'] == 'LONG']['position_value_usd'].sum()
short_mv = portfolio[portfolio['side'] == 'SHORT']['position_value_usd'].sum()

total_pnl = portfolio['unrealized_pnl_usd'].sum()
num_positions = len(portfolio)
num_long = len(portfolio[portfolio['side'] == 'LONG'])
num_short = len(portfolio[portfolio['side'] == 'SHORT'])

# 2. PORTFOLIO BETA (most important for factor neutrality!)
portfolio['position_weight'] = portfolio['position_value_usd'] / total_gmv
portfolio_beta = (portfolio['beta'] * portfolio['position_weight'].abs()).sum()

# net beta
portfolio_beta_net = (portfolio['beta'] * portfolio['position_weight']).sum()

# dollar-weighted beta
portfolio['dollar_weight'] = portfolio['position_value_usd'] / total_nmv  # signed
portfolio_beta_dollar = (portfolio['beta'] * portfolio['dollar_weight']).sum()

In [60]:
# 3. SECTOR EXPOSURE
sector_exposure = portfolio.groupby('sector').agg({
    'position_value_usd': ['sum', lambda x: x.abs().sum()],  # net, gross
    'unrealized_pnl_usd': 'sum',
    'stock_id': 'count'
}).round(4)

sector_exposure.columns = ['Net_Exposure_USD', 'Gross_Exposure_USD', 'PnL_USD', 'Num_Positions']
sector_exposure['Net_Pct_GMV'] = (sector_exposure['Net_Exposure_USD'] / total_gmv * 100).round(4)
sector_exposure['Gross_Pct_GMV'] = (sector_exposure['Gross_Exposure_USD'] / total_gmv * 100).round(4)

# 4. COUNTRY EXPOSURE
country_exposure = portfolio.groupby('country').agg({
    'position_value_usd': ['sum', lambda x: x.abs().sum()],
    'unrealized_pnl_usd': 'sum',
    'stock_id': 'count'
}).round(4)
country_exposure.columns = ['Net_Exposure_USD', 'Gross_Exposure_USD', 'PnL_USD', 'Num_Positions']
country_exposure['Net_Pct_GMV'] = (country_exposure['Net_Exposure_USD'] / total_gmv * 100).round(4)
country_exposure['Gross_Pct_GMV'] = (country_exposure['Gross_Exposure_USD'] / total_gmv * 100).round(4)

# 5. CURRENCY EXPOSURE (pre-hedge)
currency_exposure = portfolio.groupby('currency').agg({
    'position_value_usd': ['sum', lambda x: x.abs().sum()],
    'stock_id': 'count'
}).round(4)
currency_exposure.columns = ['Net_Exposure_USD', 'Gross_Exposure_USD', 'Num_Positions']
currency_exposure['Net_Pct_GMV'] = (currency_exposure['Net_Exposure_USD'] / total_gmv * 100).round(4)
currency_exposure['Gross_Pct_GMV'] = (currency_exposure['Gross_Exposure_USD'] / total_gmv * 100).round(4)

In [63]:
# 6. CONCENTRATION RISK
concentration_threshold = 0.05  # 5% of GMV
concentrated_positions = portfolio[portfolio['position_weight'] > concentration_threshold]

# 7. LIQUIDITY RISK
# Position size relative to average daily volume
portfolio['days_to_unwind'] = (
    portfolio['posn_shares'].abs() / portfolio['avg_daily_volume']
).round(4)

liquidity_threshold = 3  # Flag if takes >5 days to unwind
illiquid_positions = portfolio[portfolio['days_to_unwind'] > liquidity_threshold]

# 8. BETA OUTLIERS
beta_threshold = 2.0
high_beta = portfolio[portfolio['beta'].abs() > beta_threshold]

# 9. LARGEST P&L CONTRIBUTORS
top_winners = portfolio.nlargest(5, 'unrealized_pnl_usd')[
    ['name', 'ticker', 'sector', 'unrealized_pnl_usd', 'position_value_usd']
]
top_losers = portfolio.nsmallest(5, 'unrealized_pnl_usd')[
    ['name', 'ticker', 'sector', 'unrealized_pnl_usd', 'position_value_usd']
]

# 10. LONG VS SHORT IMBALANCE
long_short_ratio = abs(long_mv / short_mv) if short_mv != 0 else float('inf')

In [64]:
def generate_daily_risk_report(portfolio):
    """
    Generate formatted daily risk report for trading staff
    """
    
    print("=" * 80)
    print("DAILY RISK REPORT - FACTOR NEUTRAL GLOBAL EQUITIES")
    print(f"Report Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}")
    print("=" * 80)
    
    # EXECUTIVE SUMMARY
    print("\n>>> EXECUTIVE SUMMARY")
    print(f"Portfolio Beta: {portfolio_beta:.3f}")
    print(f"Net Beta: {portfolio_beta_net:.3f}")
    print(f"Dollar-Weighted Beta: {portfolio_beta_dollar:.3f}")
    print(f"Net/Gross Ratio: {net_gross_ratio:.2%}")
    print(f"Long/Short Ratio: {long_short_ratio:.2f}x")
    
    # Red flags
    print("\n>>> KEY RISKS IDENTIFIED")
    if abs(portfolio_beta) > 0.1:
        print(f"⚠️  Portfolio beta {portfolio_beta:.3f} exceeds neutral threshold (±0.1)")
    if abs(net_gross_ratio) > 0.05:
        print(f"⚠️  Net/Gross ratio {net_gross_ratio:.2%} indicates directional bias")
    if len(concentrated_positions) > 0:
        print(f"⚠️  {len(concentrated_positions)} positions exceed 5% concentration threshold")
    if len(illiquid_positions) > 0:
        print(f"⚠️  {len(illiquid_positions)} positions would take >5 days to unwind")
    if len(high_beta) > 0:
        print(f"⚠️  {len(high_beta)} positions have extreme beta (|β| > 2.0)")
    
    # PORTFOLIO OVERVIEW
    print("\n" + "=" * 80)
    print("PORTFOLIO OVERVIEW")
    print("=" * 80)
    print(f"Gross Market Value:    ${total_gmv:,.0f}")
    print(f"Net Market Value:      ${total_nmv:,.0f}")
    print(f"Long Market Value:     ${long_mv:,.0f}")
    print(f"Short Market Value:    ${short_mv:,.0f}")
    print(f"Unrealized P&L:        ${total_pnl:,.0f}")
    print(f"Total Positions:       {num_positions} ({num_long} long, {num_short} short)")
    
    # EXPOSURE ANALYSIS
    print("\n" + "=" * 80)
    print("SECTOR EXPOSURE")
    print("=" * 80)
    print(sector_exposure.sort_values('Gross_Pct_GMV', ascending=False).to_string())
    
    print("\n" + "=" * 80)
    print("COUNTRY EXPOSURE")
    print("=" * 80)
    print(country_exposure.sort_values('Gross_Pct_GMV', ascending=False).to_string())
    
    print("\n" + "=" * 80)
    print("CURRENCY EXPOSURE")
    print("=" * 80)
    print(currency_exposure.sort_values('Gross_Pct_GMV', ascending=False).to_string())
    
    # RISK FLAGS
    print("\n" + "=" * 80)
    print("RISK FLAGS - CONCENTRATED POSITIONS (>5% GMV)")
    print("=" * 80)
    if len(concentrated_positions) > 0:
        print(concentrated_positions[['name', 'ticker', 'sector', 'pct_of_gmv', 
                                      'position_value_usd']].to_string(index=False))
    else:
        print("No concentrated positions.")
    
    print("\n" + "=" * 80)
    print("RISK FLAGS - ILLIQUID POSITIONS (>5 days to unwind)")
    print("=" * 80)
    if len(illiquid_positions) > 0:
        print(illiquid_positions[['name', 'ticker', 'days_to_unwind', 
                                  'avg_daily_volume', 'posn_shares']].to_string(index=False))
    else:
        print("No illiquid positions.")
    
    print("\n" + "=" * 80)
    print("RISK FLAGS - EXTREME BETA (|β| > 2.0)")
    print("=" * 80)
    if len(high_beta) > 0:
        print(high_beta[['name', 'ticker', 'beta', 'position_value_usd', 
                        'sector']].to_string(index=False))
    else:
        print("No extreme beta positions.")
    
    # P&L ATTRIBUTION
    print("\n" + "=" * 80)
    print("TOP 5 P&L CONTRIBUTORS (Winners)")
    print("=" * 80)
    print(top_winners.to_string(index=False))
    
    print("\n" + "=" * 80)
    print("TOP 5 P&L DETRACTORS (Losers)")
    print("=" * 80)
    print(top_losers.to_string(index=False))
    
    print("\n" + "=" * 80)
    print("END OF REPORT")
    print("=" * 80)

# Generate the report
generate_daily_risk_report(portfolio)

DAILY RISK REPORT - FACTOR NEUTRAL GLOBAL EQUITIES
Report Date: 2025-10-20 22:29

>>> EXECUTIVE SUMMARY
Portfolio Beta: 1.229
Net Beta: -0.034
Dollar-Weighted Beta: 1.613
Net/Gross Ratio: -2.11%
Long/Short Ratio: 0.96x

>>> KEY RISKS IDENTIFIED
⚠️  Portfolio beta 1.229 exceeds neutral threshold (±0.1)
⚠️  1 positions would take >5 days to unwind
⚠️  368 positions have extreme beta (|β| > 2.0)

PORTFOLIO OVERVIEW
Gross Market Value:    $3,309,227,080
Net Market Value:      $-69,776,056
Long Market Value:     $1,619,725,512
Short Market Value:    $-1,689,501,568
Unrealized P&L:        $2,752,280
Total Positions:       3000 (1468 long, 1532 short)

SECTOR EXPOSURE
                            Net_Exposure_USD  Gross_Exposure_USD       PnL_USD  Num_Positions  Net_Pct_GMV  Gross_Pct_GMV
sector                                                                                                                   
Financials                     -1.802249e+06        7.715557e+08 -5.537616e+05        